## User Inputs:
Filename <br>
Sample geometry

In [ ]:
filename = "SQUIDdata-ppblsmo/ppblsmo05-MvsH-ZFC-100Oewarm-10K-20170124.rso.dat"
samplename = 'ppblsmo05'
width = 3. #mm
height = 5. #mm
thickness = 9. #nm

In [ ]:
#Imports, setting, etc.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy import interpolate
from scipy import optimize

#function to read in data from a MultiVu file
def read(fname):
    #SEPARATE HEADER FROM DATA
    headerline = ''
    nline = 0
    inputfile = open(fname,'r')
    while headerline != "[Data]\n":
        headerline = inputfile.readline()
        nline += 1
    inputfile.close()
    #READ IN DATA FILE, masked. Not all of them will be filled. The mask has values of True where data is missing
    arr = np.genfromtxt(fname, delimiter=',', skip_header=nline, names=True, usemask=True, dtype=None)
    return arr

In [ ]:
data = read(filename)
#output data columns
data.dtype.names

## Checking the data to make sure it's good quality

In [ ]:
#check that we are varying the right variables over time
plt.figure()
plt.plot(data['Field_Oe'])
plt.plot(data['Temperature_K'])

In [ ]:
plt.figure()
plt.plot(data['Field_Oe'],data['Long_Moment_emu'])

In [ ]:
plt.figure()
plt.plot(data['Long_Reg_Fit'],'.')

Filter out curves with bad fits if necessary

In [ ]:
filt = np.ma.where(data['Long_Reg_Fit'] < .85)
#filter out values with bad fit values
data[filt] = np.ma.masked

TODO: inspect raw data to see why fit failed

## Separate out field direction sweeps to manually determine saturation field

In [ ]:
nextdat = data[1:]
data['Field_Oe'][0]-nextdat['Field_Oe'][0]
where = np.ma.where(data['Field_Oe'][:-1] > nextdat['Field_Oe'])
maxfield = where[0][0]
minfield = where[0][-1] + 2 #sweeps include repeats of fields

virgin = data[:maxfield]
sweepdown = data[maxfield:minfield]
sweepup = data[minfield:]

plt.figure()
plt.plot(virgin['Field_Oe'], virgin['Long_Moment_emu'],'b.-')
plt.plot(sweepdown['Field_Oe'], sweepdown['Long_Moment_emu'],'r.-')
plt.plot(sweepup['Field_Oe'], sweepup['Long_Moment_emu'],'g.-')

In [ ]:
#"saturation" region is determined manually
uppersatfield = 3500
lowersatfield = -3500

upper = np.ma.where(data['Field_Oe'] >= uppersatfield)
lower = np.ma.where(data['Field_Oe'] <= lowersatfield)
plt.figure()
plt.plot(data['Field_Oe'][upper],data['Long_Moment_emu'][upper])
plt.plot(data['Field_Oe'][lower],data['Long_Moment_emu'][lower])

TODO: fit both regions together with the same slope

In [ ]:
#subtracting out a linear (diamagnetic) background from the substrate
lincoefflower = np.polyfit(data['Field_Oe'][lower], data['Long_Moment_emu'][lower], 1)
#TODO: fit both regions together
lincoeffupper = np.polyfit(data['Field_Oe'][upper], data['Long_Moment_emu'][upper], 1)

In [ ]:
#currently only using the middle region (lower) since it has the least temperature drift
backgroundslope = lincoefflower[0]
subtracted = data['Long_Moment_emu']-backgroundslope*data['Field_Oe']

volume = width/10. * height/10. * thickness/1.E7 #cm^3
moment = subtracted/volume #emu/cc

virg = moment[:maxfield]
down = moment[maxfield:minfield]
up = moment[minfield:]
plt.figure()
plt.plot(data['Field_Oe'][:maxfield], virg, 'b.-')
plt.plot(data['Field_Oe'][maxfield:minfield], down,'r.-')
plt.plot(data['Field_Oe'][minfield:], up, 'g.-')
plt.title('MvsH')
plt.xlabel('Applied Field H (Oe)')
plt.ylabel('Moment (emu/cc)')
ax = plt.gca()
ax.tick_params(direction='in')

## Extract parameters from MvsH curve and plot

In [ ]:
funcdown = interpolate.interp1d(data['Field_Oe'][maxfield:minfield], down, kind='cubic')
funcup = interpolate.interp1d(data['Field_Oe'][minfield:], up)
remanentdown = funcdown(0)
remanentup = funcup(0)
remanent = (remanentdown - remanentup)/2.
print('Remanent Field: '+str(remanent)+' Oe')

In [ ]:
plt.figure()
plt.plot(data['Field_Oe'][upper], moment[upper],'.-')

In [ ]:
plt.figure()
plt.plot(data['Field_Oe'][lower], moment[lower], '.-')

In [ ]:
satmomup = np.mean(moment[upper])
satmomdown = np.mean(moment[lower])
satmoment = (satmomup-satmomdown)/2.
print('Saturation moment: '+str(satmoment)+' emu/cm^3')

TODO: figure out actual saturation field rather than guessing

In [ ]:
Hcoercivedown = optimize.brentq(funcdown, -100, 0)
print('Coercive field (down): '+str(Hcoercivedown)+' Oe')
Hcoerciveup = optimize.brentq(funcup, 0, 100)
print('Coercive field (up): '+str(Hcoerciveup)+' Oe')
Hcoercive = (Hcoerciveup - Hcoercivedown)/2.
print ('Coercive field: '+str(Hcoercive)+' Oe')
Hexchangebias = (Hcoerciveup + Hcoercivedown)/2.
print ('Exchange bias field: '+str(Hexchangebias)+' Oe')

In [ ]:
plt.figure()
plt.plot(data['Field_Oe'][maxfield:minfield], down,'r.-',label='Sweep down')
plt.plot(data['Field_Oe'][minfield:], up, 'g.-',label='Sweep up')
plt.plot(0,remanentdown,'k*',label='Remanent field')
plt.plot(0,remanentup,'k*')
plt.plot(Hcoercivedown,0,'kx',label='Coercive field')
plt.plot(Hcoerciveup,0,'kx')
plt.hlines(satmomup,Hexchangebias,data['Field_Oe'][maxfield],linestyles='dashed',label='Saturation magnetization')
plt.hlines(satmomdown,data['Field_Oe'][minfield],Hexchangebias,linestyles='dashed')
plt.vlines(Hexchangebias,satmomdown,satmomup,linestyles='dotted',label='Exchange bias')
plt.title('MvsH: '+samplename)
plt.xlabel('Applied Field (Oe)')
plt.ylabel('Moment (emu/cc)')
ax = plt.gca()
plt.legend(loc=4)
ax.tick_params(direction='in')